In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from logipy import *
a = Underscore()

In [4]:
u, n = Universe().and_namespace()

n.foo[a.bla] = True
n.bar[a.bli] = True
n.bla[a.A] = n.foo(a.A) | n.bar(a.A)

print(u, "\n")
u.query(n.bla(a.X))

AttributeError: 'Variable' object has no attribute 'set_scope'

In [244]:
u, n = Universe().and_namespace()

n.parent[a.alice, a.bob] = True
n.parent[a.alice, a.charlie] = True
n.sibling[a.X, a.Y] = n.parent(a.P, a.X) & n.parent(a.P, a.Y)

print(u, "\n")
u.query(n.sibling(a, a.V))
# U: bob, V: bob
# U: bob, V: charlie
# U: charlie, V: bob
# U: charlie, V: charlie

[autoreload of logipy failed: Traceback (most recent call last):
  File "/usr/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 247, in check
    superreload(m, reload, self.old_objects)
ValueError: __init__() requires a code object with 0 free vars, not 1
]


AttributeError: 'Variable' object has no attribute 'set_scope'

In [243]:
t = a.bla(a.X, a.foo(a.Y, a.Z, a))
O = set()
t.has_occurence(a.Y)

True

In [7]:
class strset(set):
    def __str__(self):
        return "{" + ", ".join(f"({k}, {v})" for k,v in self) + "}"

def martelli_montanari(E):
    if len(E) == 0:
        return E
    
    did_a_thing = True
    tried = strset()
    
    while True:
        print(f"\nIterate: {E}")
        untried = E - tried
        if len(untried) == 0: break
        (A, B) = untried.pop()
        E.remove((A, B))
        did_a_thing = True  # Assume and unset later
        print(f"Picked ({A}, {B})")
        
        if not isinstance(A, Variable) and isinstance(B, Variable):
            # switch
            print(f"Switch ({A}, {B})")
            E.add((B, A))
        elif A == B:
            # remove (do nothing)
            print(f"Remove ({A}, {B})")
            pass
        elif isinstance(A, BasicTerm) and isinstance(B, BasicTerm):
            # peel
            if A.name == B.name and len(A.children) == len(B.children):
                print(f"Peel ({A}, {B})")
                E.update(zip(A.children, B.children))
            else:
                raise Exception("Conflict")
        elif isinstance(A, Variable) and A != B:
            # substitute
            print(f"Substitute ({A}, {B})")
            if B.has_occurence(A):
                raise Exception("Occurs check")
            # While elegant, this is somewhat slow...
            replaced_E = strset((X.replace(A, B), Y.replace(A, B)) for (X,Y) in E)
            assert len(replaced_E) == len(E)
            if E == replaced_E:
                print("Nothing changed while replacing...")
                did_a_thing = False
            else:
                print(f"{E}\n   ==   \n{replaced_E}")
                E = replaced_E
            E.add((A, B))  # Add it back
        else:
            did_a_thing = False
        
        if did_a_thing:
            tried.clear()
        else:
            # Add it back
            E.add((A, B))
            tried.add((A, B))
            print(f"Couldn't do a thing, tried = {tried})")
    
    return E

In [9]:
str(martelli_montanari(strset({(a.Y, a.bar(a.lala, a.lolo)), (a.Y, a.X), (a.bar(a.A, a.B), a.X)})))


Iterate: {(Y, X), (bar(A, B), X), (Y, bar(lala, lolo))}
Picked (Y, X)
Substitute (Y, X)
{(bar(A, B), X), (Y, bar(lala, lolo))}
   ==   
{(bar(A, B), X), (X, bar(lala, lolo))}

Iterate: {(bar(A, B), X), (Y, X), (X, bar(lala, lolo))}
Picked (bar(A, B), X)
Switch (bar(A, B), X)

Iterate: {(Y, X), (X, bar(lala, lolo)), (X, bar(A, B))}
Picked (Y, X)
Substitute (Y, X)
Nothing changed while replacing...
Couldn't do a thing, tried = {(Y, X)})

Iterate: {(Y, X), (X, bar(lala, lolo)), (X, bar(A, B))}
Picked (X, bar(lala, lolo))
Substitute (X, bar(lala, lolo))
{(Y, X), (X, bar(A, B))}
   ==   
{(bar(lala, lolo), bar(A, B)), (Y, bar(lala, lolo))}

Iterate: {(bar(lala, lolo), bar(A, B)), (X, bar(lala, lolo)), (Y, bar(lala, lolo))}
Picked (bar(lala, lolo), bar(A, B))
Peel (bar(lala, lolo), bar(A, B))

Iterate: {(Y, bar(lala, lolo)), (lolo, B), (lala, A), (X, bar(lala, lolo))}
Picked (lala, A)
Switch (lala, A)

Iterate: {(Y, bar(lala, lolo)), (lolo, B), (X, bar(lala, lolo)), (A, lala)}
Picked (A, la

'{(B, lolo), (Y, bar(lala, lolo)), (X, bar(lala, lolo)), (A, lala)}'

In [139]:
(a.X, a.bar(a.lala, a.B)) == (a.X, a.bar(a.lala, a.B))

True